## Рекомендация тарифов 

**Цель исследования** — построить модель, которая подберёт подходящий тариф.

В нашем распоряжении данные о поведении клиентов, которые уже перешли на новые тарифы. Необходимо проанализировать поведение клиентов и предложить пользователям тариф: «Смарт» или «Ультра».

## Описание данных

`сalls` — количество звонков <br>
`minutes` — суммарная длительность звонков в минутах <br>
`messages` — количество sms-сообщений <br>
`mb_used` — израсходованный интернет-трафик в Мб <br>
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd                     # импорт библиотек
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv') # чтение файла с данными и сохранение в df
except:
    df = pd.read_csv('C:\\Users\\User\\Downloads\\users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df.info() # в данных нет пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разделение выборок

Для обучения модели разделим данные на три выборки - обучающую, валидационную и тестовую в соотношении 3:1:1. Используем параметр stratify для сохранения изначального распределения целевых меток.

In [5]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345, stratify=df['is_ultra'])  # разделение выборок
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345) # на обучающую, валидационную и тестовую

In [6]:
features_train = df_train.drop(['is_ultra'], axis=1)     # признаки для обучающей выборки                        
target_train = df_train['is_ultra']                      # целевой признак для обучающей выборки   

features_valid = df_valid.drop(['is_ultra'], axis=1)     # признаки для валидационной выборки 
target_valid = df_valid['is_ultra']                      # целевой признак для валидационной выборки 

features_test = df_test.drop(['is_ultra'], axis=1)       # признаки для тестовой выборки 
target_test = df_test['is_ultra']                        # целевой признак для тестовой выборки 

print(features_train.shape)    # размер выборок
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)


(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


In [7]:
print(f"Количество строк в target_train по классам: {np.bincount(target_train)}") # проверим равномерно ли разбиты данные 
print(f"Количество строк в target_valid по классам: {np.bincount(target_valid)}")  # по классам
print(f"Количество строк в target_test по классам: {np.bincount(target_test)}")

Количество строк в target_train по классам: [1337  591]
Количество строк в target_valid по классам: [445 198]
Количество строк в target_test по классам: [447 196]


## Исследование моделей

### Решающее дерево

In [8]:
model_1 = DecisionTreeClassifier(random_state=12345)

In [9]:
parametrs = { 'max_depth': range (1,15, 1),  # параметры для поиска лучшей модели
              'min_samples_leaf': range (1,10),
              'min_samples_split': range (2,10,2)}

In [10]:
grid_tree = GridSearchCV(model_1, parametrs, cv=5)   
grid_tree.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'max_depth': range(1, 15),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10, 2)})

In [11]:
grid_tree.best_params_   # выводим результат с лучшими параметрами

{'max_depth': 6, 'min_samples_leaf': 7, 'min_samples_split': 2}

In [12]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6, min_samples_leaf=7, min_samples_split=2)
model.fit(features_train, target_train) # обучение  
predictions_valid = model.predict(features_valid) # предсказания на валидационной выборке
print(accuracy_score(target_valid, predictions_valid)) 

0.7869362363919129


Лучшее значение accuracy = 0.78 показывает модель со значениями max_depth = 6, min_samples_leaf = 7, min_samples_split = 2

### Случайный лес

In [13]:
model_rf = RandomForestClassifier(random_state=12345)

In [14]:
parametrs = { 'n_estimators': range (10, 51, 10),   # параметры для поиска лучшей модели
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }

In [15]:
grid = GridSearchCV(model_rf, parametrs, cv=5)   
grid.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=12345),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(1, 8),
                         'min_samples_split': range(2, 10, 2),
                         'n_estimators': range(10, 51, 10)})

In [16]:
grid.best_params_   # выводим результат с лучшими параметрами

{'max_depth': 9,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 10}

In [17]:
# обучение модели с лучшими параметрами
model_RandomForest = RandomForestClassifier(random_state=12345, max_depth=9, n_estimators=10, min_samples_leaf=2,
                    min_samples_split=2) 
model_RandomForest.fit(features_train, target_train) # обучение  
predictions_valid = model_RandomForest.predict(features_valid) # предсказания на валидационной выборке
print(accuracy_score(target_valid, predictions_valid))

0.8102643856920684


Лучшее значение accuracy_score равное 0,81 достигается при n_estimators = 10, max_depth = 9, min_samples_leaf = 2, min_samples_split = 2.

### Логистическая регрессия

In [18]:
model_lr = LogisticRegression(random_state=12345) 

In [19]:
parametrs = { 'max_iter': [100, 1001, 1000],   # параметры для поиска лучшей модели
              'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

In [20]:
grid = GridSearchCV(model_lr, parametrs)   
grid.fit(features_train, target_train)

GridSearchCV(estimator=LogisticRegression(random_state=12345),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'max_iter': [100, 1001, 1000]})

In [21]:
grid.best_params_ # выводим результат с лучшими параметрами

{'C': 0.1, 'max_iter': 100}

In [22]:
model_LogisticRegression = LogisticRegression(random_state=12345, max_iter=100, C=0.1) 
model_LogisticRegression.fit(features_train, target_train) #  обучение модели
LogisticRegression_predict = model_LogisticRegression.predict(features_valid)#предсказания на валидационной выборке
print(model_LogisticRegression.score(features_valid, target_valid))

0.7371695178849145


Лучшее значение accuracy равное 0.73 достигается при значении C = 0.1, max_iter = 100

## Проверка на тестовой выборке

Проверка решающего дерева на тестовой выборке. 

In [23]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6, min_samples_leaf=7, min_samples_split=2)
model.fit(features_train, target_train) 
test_predictions = model.predict(features_test)
accuracy_test = accuracy_score(target_test.values, test_predictions)
accuracy_test

0.7807153965785381

Проверка случайного леса на тестовой выборке.

In [24]:
model_RandomForest = RandomForestClassifier(random_state=12345, max_depth=9, n_estimators=10, min_samples_leaf=2,
                    min_samples_split=2)
model_RandomForest.fit(features_train, target_train) 
test_predictions = model_RandomForest.predict(features_test)
accuracy_test = accuracy_score(target_test.values, test_predictions)
accuracy_test 

0.8118195956454122

Проверка логистической регрессии на тестовой выборке.

In [25]:
model_LogisticRegression = LogisticRegression(random_state=12345, max_iter=100, C=0.1)
model_LogisticRegression.fit(features_train, target_train) 
test_predictions = model_LogisticRegression.predict(features_test)
accuracy_test = accuracy_score(target_test.values, test_predictions)
accuracy_test

0.7480559875583204

Самое высокое значение accuracy = 0.81 на тестовой выборке у модели случайного леса.

Значения accuracy для весх моделей на обучающей и валидационной выборках примерно равны.

## Проверка на вменяемость

Проверим на тренировочных данных заполнение ответом "Смарт".

In [26]:
dummy_clf = DummyClassifier(strategy = "constant", random_state=12345, constant=[0])
dummy_clf.fit(features_train, target_train)

dummy_clf.predict(features_train)

dummy_clf.score(features_train, target_train)

0.6934647302904564

Проверим на тренировочных данных заполнение случайными ответами.

In [27]:
dummy_clf = DummyClassifier(strategy = "uniform", random_state=12345)
dummy_clf.fit(features_train, target_train)

dummy_clf.predict(features_train)

dummy_clf.score(features_train, target_train)

0.4880705394190871

Проверка на вменяемость показала, что модели Случайный лес и Решающее дерево предсказывают лучше, чем заполнение случайными данными или частотным значением. 

| Имя модели    | Значение accuracy        | 
|:------------- |:---------------:| 
| Случайный лес      | 0.81 | 
| Решающее дерево        | 0.78        | 
| Логистическая регрессия | 0.74        | 

Использование модели Случайный лес предпочтительнее.